In [15]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
import os
import langchain
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
from langchain_openai import ChatOpenAI
os.environ["OPENAI_API_KEY"]='sk-'
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [ ]:
!pip install arxiv
!pip install -U langchain
!pip install -U langchain_community
!pip install -U langchain_openai
!pip install faiss-cpu
!pip install pypdf
!pip install transformers
!pip install wikipedia

In [16]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [17]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information ")


In [18]:
tools=[wiki,arxiv,retriever_tool]
llm=ChatOpenAI(model='gpt-3.5-turbo',temperature=0)


In [19]:
!pip install langchainhub
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

Defaulting to user installation because normal site-packages is not writeable


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [20]:

### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor
agent_executor.invoke({"input":"What are neural networks in deep learning"})




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Neural networks in deep learning`


Page: Deep learning
Summary: Deep learning is the subset of machine learning methods based on artificial neural networks (ANNs) with representation learning. The adjective "deep" refers to the use of Neural networks in deep learning are a subset of machine learning methods based on artificial neural networks (ANNs) with representation learning. The term "deep" in deep learning refers to the use of multiple layers in the neural network architecture. These networks are designed to automatically learn and improve from experience without being explicitly programmed.

> Finished chain.


{'input': 'What are neural networks in deep learning',
 'output': 'Neural networks in deep learning are a subset of machine learning methods based on artificial neural networks (ANNs) with representation learning. The term "deep" in deep learning refers to the use of multiple layers in the neural network architecture. These networks are designed to automatically learn and improve from experience without being explicitly programmed.'}